In [1]:
import pandas as pd

# Read CSV files
df1 = pd.read_csv('./transfermarkt_cleaned/clubs.csv')
df2 = pd.read_csv('./transfermarkt_cleaned/players.csv')
df3 = pd.read_csv('./transfermarkt_cleaned/appearances.csv')
df4 = pd.read_csv('./transfermarkt_cleaned/games.csv')


In [2]:
from sqlalchemy import create_engine

# Create SQLite in-memory database
engine = create_engine('sqlite:///:memory:')

In [3]:
# Add the CSVs to SQLite database as tables
df1.to_sql('clubs', engine, index=False)
df2.to_sql('players', engine, index=False)
df3.to_sql('appearances', engine, index=False)
df4.to_sql('games', engine, index=False)

2419

Describe the tables.

In [6]:
query = """
PRAGMA table_info([games]);
"""

result = pd.read_sql(query, engine)
print(result)

    cid                    name    type  notnull dflt_value  pk
0     0                 game_id  BIGINT        0       None   0
1     1          competition_id    TEXT        0       None   0
2     2                  season  BIGINT        0       None   0
3     3                   round    TEXT        0       None   0
4     4                    date    TEXT        0       None   0
5     5            home_club_id  BIGINT        0       None   0
6     6            away_club_id  BIGINT        0       None   0
7     7         home_club_goals  BIGINT        0       None   0
8     8         away_club_goals  BIGINT        0       None   0
9     9      home_club_position   FLOAT        0       None   0
10   10      away_club_position   FLOAT        0       None   0
11   11  home_club_manager_name    TEXT        0       None   0
12   12  away_club_manager_name    TEXT        0       None   0
13   13                 stadium    TEXT        0       None   0
14   14              attendance   FLOAT 

In [7]:
query = """
PRAGMA table_info([appearances]);
"""

result = pd.read_sql(query, engine)
print(result)

    cid                    name    type  notnull dflt_value  pk
0     0           appearance_id    TEXT        0       None   0
1     1                 game_id  BIGINT        0       None   0
2     2               player_id  BIGINT        0       None   0
3     3          player_club_id  BIGINT        0       None   0
4     4  player_current_club_id  BIGINT        0       None   0
5     5                    date    TEXT        0       None   0
6     6             player_name    TEXT        0       None   0
7     7          competition_id    TEXT        0       None   0
8     8            yellow_cards  BIGINT        0       None   0
9     9               red_cards  BIGINT        0       None   0
10   10                   goals  BIGINT        0       None   0
11   11                 assists  BIGINT        0       None   0
12   12          minutes_played  BIGINT        0       None   0


In [8]:
query = """
PRAGMA table_info([players]);
"""

result = pd.read_sql(query, engine)
print(result)

   cid                         name    type  notnull dflt_value  pk
0    0                    player_id  BIGINT        0       None   0
1    1                         name    TEXT        0       None   0
2    2                  last_season  BIGINT        0       None   0
3    3              current_club_id  BIGINT        0       None   0
4    4                 height_in_cm   FLOAT        0       None   0
5    5          market_value_in_eur   FLOAT        0       None   0
6    6  highest_market_value_in_eur   FLOAT        0       None   0


In [9]:
query = """
PRAGMA table_info([clubs]);
"""

result = pd.read_sql(query, engine)
print(result)

   cid                     name    type  notnull dflt_value  pk
0    0                  club_id  BIGINT        0       None   0
1    1                     name    TEXT        0       None   0
2    2  domestic_competition_id    TEXT        0       None   0
3    3             stadium_name    TEXT        0       None   0
4    4      net_transfer_record    TEXT        0       None   0
5    5              last_season  BIGINT        0       None   0


--- STARTING THE QUESTIONS PART ---

Q1. Finding the average height of players in the leagues.

In [12]:
query = """
    SELECT c.domestic_competition_id AS 'league', AVG(p.height_in_cm)
    FROM clubs c 
    JOIN players p 
    ON p.current_club_id = c.club_id
    GROUP BY c.domestic_competition_id;
"""

result = pd.read_sql(query, engine)
print(result)

  league  AVG(p.height_in_cm)
0    ES1           181.649624
1    FR1           181.836834
2    GB1           182.731034
3    IT1           184.190409
4     L1           184.665535
5    TR1           182.055306


Here I wanted ChatGPT to generate me some questions.

Q2. Determine which players had the biggest decrease in market value over their career (highest market value minus the current market value) limit by 5.

In [41]:
query = """
    SELECT p.name, 
        (p.highest_market_value_in_eur - (
                SELECT MIN(p2.market_value_in_eur) 
                FROM players p2 
                WHERE p2.player_id = p.player_id
            )
        ) AS 'Value Difference'
    FROM players p
    ORDER BY [Value Difference] DESC
    LIMIT 5;
"""

result = pd.read_sql(query, engine)
print(result)

                name  Value Difference
0        Eden Hazard       145000000.0
1       Lionel Messi       145000000.0
2  Philippe Coutinho       140000000.0
3         Sadio Mané       125000000.0
4  Antoine Griezmann       125000000.0


Q3. Which referee has given the most red cards in every league?

In [49]:
query = """
    WITH red_cards AS(
        SELECT c.domestic_competition_id, g.referee, SUM(app.red_cards) AS 'sum_red_cards'
        FROM games g
        JOIN appearances app
        ON g.game_id = app.game_id
        JOIN clubs c
        ON c.club_id = g.home_club_id
        GROUP BY c.domestic_competition_id, g.referee
    )
    SELECT rc.domestic_competition_id, rc.referee, MAX(rc.sum_red_cards) AS 'Red Cards'
    FROM red_cards rc
    GROUP BY rc.domestic_competition_id
    ORDER BY [Red Cards] DESC;
"""

result = pd.read_sql(query, engine)
print(result)

  domestic_competition_id                  referee  Red Cards
0                     FR1          Mathieu Vernice          7
1                     ES1  Carlos del Cerro Grande          7
2                     TR1            Mete Kalkavan          6
3                     IT1             Davide Massa          5
4                      L1              Tobias Welz          4
5                     GB1           Anthony Taylor          4


Q4. Which players had the most red cards divided by the sum of games they have played, limit by 5, all leagues?

In [82]:
query = """
    WITH player_stats AS (
        SELECT p.name, p.player_id, COUNT(app.appearance_id) AS 'games_played', SUM(app.red_cards) AS 'red_cards'
        FROM players p
        JOIN appearances app
        ON p.player_id = app.player_id
        GROUP BY p.player_id
    )
    SELECT ps.name, ROUND(1.0 * ps.red_cards / ps.games_played, 5) AS 'ratio', ps.red_cards, ps.games_played
    FROM player_stats ps
    WHERE ps.red_cards > 0
    ORDER BY ratio DESC
    LIMIT 5;
"""

result = pd.read_sql(query, engine)
print(result)

                   name    ratio  red_cards  games_played
0        Jorge Herrando  1.00000          1             1
1  Hasan Emre Yesilyurt  1.00000          1             1
2        Luís Maximiano  0.33333          1             3
3    Yaroslav Rakitskyi  0.25000          2             8
4    Nathaniel Chalobah  0.20000          1             5


Q5. Which clubs had the least number of yellow cards and red cards combined in the latest season? Rank the top 5.

Q6. Identify players who have scored in at least 5 consecutive appearances. 

Q7. Which stadium had the highest average attendance in the latest season? Include stadium name and the average attendance.

Q8. Which manager had the most games where their team scored more than 3 goals in the latest season?